In [1]:
import pandas as pd
import string
from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


# BW1484 Core I Output

In [2]:
k_ref = 1.00020

### Load and parse the file

In [11]:
import re

# name of the file
out_file='bw1484_small.out' #$1
f = open('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/'+out_file, 'r')

## Pairs of output variables and their keywords that always occur and only once
keys = ['problem_name', 'eigen_solver', 'eq_set', 'num cells I', 'num cells J', 
        'num cells K', 'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'DoF', 
        'L2_tolerance', 'k_tolerance', 'first_group', 'last_group', 'mesh ', 'num_sets', 
        'num upscatter groups', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
        'relax_weight', 'Pn_order', 'azimuthals_octant', 'polars_octant']
vals = {key: None for key in keys}

# set flags and keys to use for items with duplicate entries
ml_flag = False
ml_keys = ['Pn_order', 'azimuthals_octant', 'polars_octant']
ml_doubles = {key: None for key in ml_keys}
ml_cnt = 0

up_db = False
doubles = {'tolerance' : None}

for line in f:
    
    # check for the duplicate items before processing the next stuff
    if ml_flag == True: 
        if ml_cnt != 3:
            for ml_key in ml_doubles:
                if re.search(ml_key, line):
                    if ml_doubles[ml_key] == None:
                        data = line.split(ml_key)
                        ml_doubles[ml_key] = data[1].strip() 
                        ml_cnt = ml_cnt + 1
        else:
            ml_flag = False
    if up_db == True:
        if re.search('tolerance', line):
            if doubles['tolerance'] == None:
                data = line.split('tolerance')
                doubles['tolerance'] = data[1].strip() 
                up_db = False
        
    for key in vals: 
        # check for things to set the flags
        if re.search('multilevel_quad_db', line):
            ml_flag = True
        if re.search('upscatter_db', line):
            up_db = True   
        # now look for the rest of the keys
        if re.search(key, line):
            data = line.split(key)
            vals[key] = data[1].strip()  
        
# set things we want in the spreadsheet
vals['num_groups'] = int(vals['last_group']) - int(vals['first_group']) + 1
vals['ml_Pn'] = ml_doubles['Pn_order']
vals['ml_az'] = ml_doubles['azimuthals_octant']
vals['ml_oct'] = ml_doubles['polars_octant']
vals['up_tol'] = doubles['tolerance']

print vals
# ones needing special handling: 'Pn_order', 'azimuthals_octant', 'polars_octant', 'tolerance', 

# if multilevel_quad_db,
# Pn_order,
# azimuthals_octant,
# polars_octant,
# if upscatter_db; tolerance,


{'num_sets': '2', 'Pn_order': '3', 'last_group': '7', 'ml_oct': '2', 'relax_weight': '1', 'up_tol': '0.001', 'num_groups': 8, 'relax_count': '1', 'mg_preconditioner': 'Multilevel', 'k_tolerance': '0.001', 'azimuthals_octant': '6', 'num upscatter groups': '4', 'eigen_solver': 'rayleigh_quotient', 'num_v_cycles': '1', 'L2_tolerance': '0.001', 'ml_Pn': '1', 'eq_set': 'ld', 'ml_az': '2', 'num_z_blocks': '7', 'num cells I': '120', 'num cells J': '120', 'num cells K': '84', 'mesh ': '0', 'problem_name': 'out_bw1484_RQI_sn', 'num_blocks_i': '16', 'num_blocks_j': '16', 'DoF': '4954521600', 'first_group': '0', 'polars_octant': '12'}
